In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install torch

In [ ]:
!pip install transformers==3

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
train = pd.read_csv("/content/drive/My Drive/crawling_20211104.csv")

In [ ]:
df_shuffled=train.iloc[np.random.permutation(train.index)].reset_index(drop=True)

In [ ]:
train = df_shuffled.copy()

label = []
for s in train['scores']:
  if s == "width: 100%":
    label.append(0)
  elif s == "width: 80%":
    label.append(1)
  else:
    label.append(2)

train['label'] = label

train_df = train[:-8000]
val_df = train[-8000:]

In [ ]:
using_train_data = train_df[['rvs', 'label', 'categories', 'retypes',
       'meta_sizes', 'meta_brights', 'meta_colors', 'meta_thicks', ]]
      #  'pur_option', 'cus_sex', 'cus_height', 'cus_weight']]

using_val_data = val_df[['rvs', 'label', 'categories', 'retypes',
       'meta_sizes', 'meta_brights', 'meta_colors', 'meta_thicks', ]]
      #  'pur_option', 'cus_sex', 'cus_height', 'cus_weight']]

In [ ]:
using_data = train[['rvs', 'label', 'retypes', 'categories', 'meta_sizes', 'meta_brights', 'meta_colors', 'meta_thicks']]

In [ ]:
meta_df = pd.DataFrame()
for c in using_data.columns.tolist()[2:]:
  dummy_cate = pd.get_dummies(using_data[c])
  meta_df = pd.concat([meta_df, dummy_cate], axis=1)

train_meta_df = meta_df[:-8000]
val_meta_df = meta_df[-8000:]

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, df, meta_df, bert_tokenizer, max_len,
                 pad, pair, only_ct=True):
      
        self.only_ct= only_ct
        self.df = df
        self.meta_df = meta_df

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        def clean_text(sent):
            sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
            return sent_clean
        
        self.sentences = [transform([clean_text(str(i[0]))]) for i in list(df.iloc[:, 0])]

    def __getitem__(self, idx):

        label = torch.tensor(self.df.iloc[idx, 1])

        if self.only_ct:
            meta = self.meta_df.iloc[idx, :]
        else:
            meta = self.meta_df.iloc[idx, 3:9]
        meta = torch.tensor(meta).float()

        return (self.sentences[idx] + (label, ), meta)

    def __len__(self):
        return (len(self.sentences))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 2
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_trains = BERTDataset(using_train_data, train_meta_df, tok, max_len, True, False)
data_tests = BERTDataset(using_val_data, val_meta_df, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_trains, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_tests, batch_size=batch_size, num_workers=2, shuffle=False)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 meta_input_dim = 21,
                 meta_hidden_dim = 64,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None,
                 use_m=True
                 ):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.use_m = use_m
        
        if self.use_m:
          self.classifier = nn.Sequential(
              nn.Linear(hidden_size + meta_input_dim, hidden_size), #hidden*2
              nn.Linear(hidden_size, num_classes)
          )

        else:
          self.classifier = nn.Sequential(
              nn.Linear(hidden_size, hidden_size),
              nn.Linear(hidden_size, num_classes)
          )
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
            
        self.meta_feature = nn.Sequential(
            nn.Linear(meta_input_dim, meta_hidden_dim),
            nn.Linear(meta_hidden_dim, meta_input_dim)
        )

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, meta_onehot):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        if self.use_m :
          meta_feature = self.meta_feature(meta_onehot)
          out = torch.cat([out, meta_feature], dim=1)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, ((token_ids, valid_length, segment_ids, label), meta) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        meta = meta.to(device)
        out = model(token_ids, valid_length, segment_ids, meta)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            torch.save(model.state_dict(), "./drive/MyDrive/dacon/bert_model.pt")
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, ((token_ids, valid_length, segment_ids, label), meta) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        meta = meta.to(device)
        out = model(token_ids, valid_length, segment_ids, meta)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))